# Flash Report — Complete Operation 1 + Report Generation

Complete Operation 1 from Flash_Report.py with Excel report generation capabilities.

**Instructions:**
1. Update file paths in the configuration cell
2. Run all cells in order
3. Check output Excel files in current directory

In [ ]:
# Imports
import pandas as pd
import numpy as np
import json
import os
from difflib import get_close_matches
from datetime import datetime

pd.set_option('display.max_columns', None)
print('Libraries imported successfully')

## Configuration - Update File Paths

In [ ]:
# REQUIRED: Update these paths to your actual files
file_path = r'REPLACE_WITH_MAIN_DATA_FILE.xlsx'  # Web Intelligence Export
source_path_ca = r'REPLACE_WITH_CA_REFERENCE_FILE.xlsx'  # CA reference (Sheet1)
source_path_us = r'REPLACE_WITH_US_REFERENCE_FILE.xlsx'  # US reference (Sheet1)

# Output configuration
output_us_file = 'Final_Report_USD_US.xlsx'
output_ca_file = 'Final_Report_Canada_CA.xlsx'
us_partners_folder = 'US_partners_report'
ca_partners_folder = 'Canada_partners_report'

print('Configuration:')
print('Main:', file_path)
print('CA Ref:', source_path_ca)
print('US Ref:', source_path_us)
print('Output US:', output_us_file)
print('Output CA:', output_ca_file)

## Helper Functions

In [ ]:
def get_default_mappings():
    return {
        'main_file_mappings': {
            'SRC_SYS_KY': ['SRC_SYS_KY', 'Src Sys Ky'],
            'CROSS_SOURCED': ['CROSS_SOURCED', 'Cross Sourced'],
            'BDE_FLAG': ['BDE_FLAG', 'Bde Flag'],
            'MSP_FLAG': ['MSP_FLAG', 'MSP Flag'],
            'REPORTING_TYPE': ['REPORTING_TYPE', 'Reporting Type'],
            'PRODUCT_LINE': ['PRODUCT_LINE', 'Product Line'],
            'RESELLER_PARTY_ID': ['RESELLER_PARTY_ID', 'Reseller Party Id'],
            'DISTRIBUTOR_PARTY_ID': ['DISTRIBUTOR_PARTY_ID', 'Distributor Party Id'],
            'FISCAL_MONTH': ['FISCAL_MONTH', 'Fiscal Month'],
            'NDP_TOTAL_USD': ['NDP_TOTAL_USD', 'Ndp Total Usd'],
            'NET_TOTAL_USD': ['NET_TOTAL_USD', 'Net Total Usd'],
            'UPFRONT_DISCOUNT_AMT_USD': ['UPFRONT_DISCOUNT_AMT_USD', 'Upfront Discount Amt Usd'],
            'BACKEND_DISCOUNT_AMT_USD': ['BACKEND_DISCOUNT_AMT_USD', 'Backend Discount Amt Usd'],
            'DATA_TYPE': ['DATA_TYPE', 'Data Type'],
            'BACKEND_DEAL_1': ['BACKEND_DEAL_1', 'Backend Deal 1'],
            'INVOICE_NUMBER': ['INVOICE_NUMBER', 'Invoice Number'],
            'HPE_SALES_ORDER_NUMBER': ['HPE_SALES_ORDER_NUMBER', 'Hpe Sales Order Number'],
            'NET_TOTAL_LC': ['NET_TOTAL_LC', 'Net Total Lc'],
            'BACKEND_DISCOUNT_AMT_LC': ['BACKEND_DISCOUNT_AMT_LC', 'Backend Discount Amt Lc'],
            'UPFRONT_DISCOUNT_AMT_LC': ['UPFRONT_DISCOUNT_AMT_LC', 'Upfront Discount Amt Lc'],
            'NDP_TOTAL_LC': ['NDP_TOTAL_LC', 'Ndp Total Lc'],
            'DISTRIBUTOR_PARTY_NAME': ['Distributor Party Name', 'DISTRIBUTOR_PARTY_NAME'],
            'RESELLER_PARTY_NAME': ['Reseller Party Name', 'RESELLER_PARTY_NAME'],
            'PRODUCT_NUMBER': ['Product Number', 'PRODUCT_NUMBER']
        }
    }

def load_column_mappings():
    try:
        with open('column_mappings.json', 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        print('[WARNING] column_mappings.json not found, using default mappings')
        return get_default_mappings()
    except json.JSONDecodeError:
        print('[ERROR] Invalid JSON in column_mappings.json, using default mappings')
        return get_default_mappings()

def find_column_match(target_column, available_columns, mappings):
    if target_column in mappings:
        for variant in mappings[target_column]:
            if variant in available_columns:
                return variant
        for variant in mappings[target_column]:
            close_matches = get_close_matches(variant, available_columns, n=1, cutoff=0.8)
            if close_matches:
                return close_matches[0]
    close_matches = get_close_matches(target_column, available_columns, n=1, cutoff=0.7)
    if close_matches:
        return close_matches[0]
    return None

def standardize_column_names(df, file_type='main'):
    mappings = load_column_mappings()
    if file_type == 'main':
        target_mappings = mappings.get('main_file_mappings', {})
    else:
        target_mappings = mappings.get('reference_file_mappings', {})
    column_mapping = {}
    available_columns = list(df.columns)
    print(f"[*] Standardizing {file_type} file columns...")
    for target_col, variants in target_mappings.items():
        matched_col = find_column_match(target_col, available_columns, target_mappings)
        if matched_col:
            column_mapping[matched_col] = target_col
            print(f"[*] Mapped '{matched_col}' -> '{target_col}'")
        else:
            print(f"[WARNING] No match found for required column: {target_col}")
    df_standardized = df.rename(columns=column_mapping)
    return df_standardized, column_mapping

def validate_main_file(df):
    df_standardized, column_mapping = standardize_column_names(df, 'main')
    required_columns = ['SRC_SYS_KY','CROSS_SOURCED','BDE_FLAG','MSP_FLAG','REPORTING_TYPE','PRODUCT_LINE','RESELLER_PARTY_ID','DISTRIBUTOR_PARTY_ID','FISCAL_MONTH','NDP_TOTAL_USD','NET_TOTAL_USD','UPFRONT_DISCOUNT_AMT_USD','BACKEND_DISCOUNT_AMT_USD','DATA_TYPE','BACKEND_DEAL_1','INVOICE_NUMBER','HPE_SALES_ORDER_NUMBER','NET_TOTAL_LC','BACKEND_DISCOUNT_AMT_LC','UPFRONT_DISCOUNT_AMT_LC','NDP_TOTAL_LC']
    missing_columns = [col for col in required_columns if col not in df_standardized.columns]
    if missing_columns:
        raise ValueError(f"Main file missing columns: {missing_columns}")
    print('[*] Main file format validation passed')
    return df_standardized

def validate_reference_file(df, file_type):
    df_standardized, column_mapping = standardize_column_names(df, 'reference')
    required_columns = ['PL','BU','TYPE','EXCLUSION_PARTY_ID','EXCLUSION_LEVEL','PG_EXCLUSION_ELIGIBLE_LIST_PARTY_ID','LOC_ID','ELICPES','PN_PL','BU_1','COMMON_PL','COMMON_PN_PL']
    missing_columns = [col for col in required_columns if col not in df_standardized.columns]
    if missing_columns:
        raise ValueError(f"{file_type} reference file missing columns: {missing_columns}")
    print(f"[*] {file_type} reference file format validation passed")
    return df_standardized

print('Helper functions defined')

## Load and Validate Input Files

In [ ]:
print('[*] Reading main data file...')
df_raw = pd.read_excel(file_path, engine='openpyxl')
df = validate_main_file(df_raw)
print(f"[*] Main data file successfully loaded. Total rows: {len(df)}")

print('[*] Reading the Reference File for CA (Sheet1) ...')
df_source_ca_raw = pd.read_excel(source_path_ca, sheet_name='Sheet1')
df_source_ca = validate_reference_file(df_source_ca_raw, 'CA')

print('[*] Reading the Reference File for US (Sheet1) ...')
df_source_us_raw = pd.read_excel(source_path_us, sheet_name='Sheet1')
df_source_us = validate_reference_file(df_source_us_raw, 'US')

print('All input files loaded and validated successfully!')

## Operation 1 — Data Filtering and Processing

In [ ]:
print('='*60)
print('OPERATION 1: BOE Data Validation and Filtering')
print('='*60)

# Filter for SRC_SYS_KY in [2032, 2866]
df_src = df[df['SRC_SYS_KY'].isin([2032, 2866])]
print(f"[*] After filtering for SRC_SYS_KY: {len(df_src)} rows")

# Remove CROSS_SOURCED == 'Y'
df_cross_sourced = df_src[df_src['CROSS_SOURCED'] != 'Y']
print(f"[*] After removing CROSS_SOURCED Y: {len(df_cross_sourced)} rows")

# Remove BDE_FLAG == 'Y' and fill N
df_bde = df_cross_sourced[df_cross_sourced['BDE_FLAG'] != 'Y'].copy()
df_bde['BDE_FLAG'] = df_bde['BDE_FLAG'].fillna('N')
print(f"[*] After removing BDE_FLAG Y: {len(df_bde)} rows")

# Remove MSP_FLAG == 'T'
df_msp = df_bde[df_bde['MSP_FLAG'] != 'T']
print(f"[*] After removing MSP_FLAG T: {len(df_msp)} rows")

# Remove REPORTING_TYPE == 'RCS' (also keep RCS separately for reference)
df_reporting = df_msp[df_msp['REPORTING_TYPE'] != 'RCS']
df_rcs = df_msp[df_msp['REPORTING_TYPE'] == 'RCS']
print(f"[*] After removing REPORTING_TYPE RCS: {len(df_reporting)} rows")
print(f"[*] RCS Data for reference: {len(df_rcs)} rows")

# Add BU/BU_Type/Scheme_Name and reset index
df_extend_columns = df_reporting.assign(BU='', BU_Type='', Scheme_Name='').reset_index(drop=True)
print(f"[*] After adding columns and resetting index: {len(df_extend_columns)} rows")

print('\nData filtering completed successfully!')